In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
%run -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.4 MB/s eta 0:00:001m49.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
from pathlib import Path

import pandas as pd
import spacy

f = Path(f"./tlg0525.tlg001.perseus-eng2.pickle")

def tokenize(df: pd.DataFrame):
    model = "en_core_web_sm"

    nlp = spacy.load(model, disable=["ner"])

    df["tokens"] = df["unannotated_strings"].apply(nlp.tokenizer)

    raw_texts = [t for t in df["unannotated_strings"]]
    annotated_texts = nlp.pipe(raw_texts, batch_size=100)

    df["nlp_docs"] = list(annotated_texts)

    return df

df = tokenize(pd.read_pickle(f))

# Bug 1

Why is the function `expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1)` returning `0.0` no matter what we pass to it?

In [3]:
from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """

    lemmata = [t for t in df['nlp_docs'].explode()]
    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(lemmata)

expected_freq_the_mainlaind = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainlaind

0.0

# Bug 1 analysis and response

# Bug 2

Why does the code below not give us the number of occurrences of the token "the"?

In [8]:
observed_freq_the = df['tokens'].str.find('the')

observed_freq_the

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3165   NaN
3166   NaN
3167   NaN
3168   NaN
3169   NaN
Name: tokens, Length: 3170, dtype: float64

# Bug 2 analysis and response

# Bug 3

How is `o11` greater than `r1`, and why is our `o12` negative?

In [5]:
node = "mainland"
collocate = "the"

def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    # the right-hand side of a slice in Python is exclusive, so we add 1 to make sure
    # we're actually getting one element to the right

    chunked_lemmata = [lemmata[i - l_size:i + r_size + 1] for i in range(0, len(lemmata))]
    
    cooccurrences = [1 for l in chunked_lemmata if w1 in l and w2 in l]

    return sum(cooccurrences)

r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate)).sum()
o12 = r1 - o11
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")

r1: 38, r2: 311516, c1: 26932, c2: 284622, o11: 63, o12: -25, o21: 26869


# Bug 3 analysis and response